In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import pygad
import wandb

from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

tf.random.set_seed(0)
np.random.seed(0)

2023-06-23 15:49:07.459785: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-23 15:49:07.520590: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-06-23 15:49:09.171019: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/macula/SMATousi/.conda/envs/mac-deep/lib/:/home/macula/SMATousi/.conda/envs/mac-deep/lib/py

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 1722120911450780791
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 22706061312
locality {
  bus_id: 1
  links {
  }
}
incarnation: 3780915305601874606
physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:01:00.0, compute capability: 8.6"
xla_global_id: 416903419
]


2023-06-23 15:49:14.776127: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-23 15:49:14.824250: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-06-23 15:49:14.842613: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-06-23 15:49:14.842682: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at

# Creating the dataset

In [2]:
num_samples = 10000
num_dimensions = 30

# Generate random samples
raw_data = np.random.rand(num_samples, num_dimensions)

outlier_indices_1 = [0,1,2,3,4,5,6]
outlyin_amount_1 = 20
outlier_indices_2 = [0,2,4,6,8,10,12]
outlyin_amount_2 = 10
outlier_indices_3 = [0,3,6,9,12,15,18]
outlyin_amount_3 = 5
outlier_indices_4 = [0,4,8,12,16,20,24]
outlyin_amount_4 = 2
outlier_indices_5 = [0,5,10,15,20,25,29]
outlyin_amount_5 = 1.1
outlier_indices_6 = [0,1,2,3,4,5,6]
outlyin_amount_6 = [20,15,10,5,2,1.1,1.05]

corrupted_data = raw_data

corrupted_data[100:120, outlier_indices_1] = outlyin_amount_1
corrupted_data[200:220, outlier_indices_2] = outlyin_amount_2
corrupted_data[300:320, outlier_indices_3] = outlyin_amount_3
corrupted_data[400:420, outlier_indices_4] = outlyin_amount_4
corrupted_data[500:520, outlier_indices_5] = outlyin_amount_5
corrupted_data[600:620, outlier_indices_6] = outlyin_amount_6

for outlier_gene_index in range(corrupted_data.shape[1]+1):
    corrupted_data[700+outlier_gene_index,:outlier_gene_index] = outlyin_amount_1
    

# print(raw_data)

In [ ]:
corrupted_data[701]

# Training the VAE

In [3]:

class Sampling(layers.Layer):
    """Uses (z_mean, z_log_var) to sample z, the vector encoding a digit."""

    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon


latent_dim = 2

encoder_inputs = keras.Input(shape=(30,))
# x = layers.Conv2D(32, 3, activation="relu", strides=2, padding="same")(encoder_inputs)
# x = layers.Conv2D(64, 3, activation="relu", strides=2, padding="same")(x)
# x = layers.Flatten()(x)
x = layers.Dense(30, activation="tanh")(encoder_inputs)
x = layers.Dense(20, activation="tanh")(x)
x = layers.Dense(18, activation="tanh")(x)
x = layers.Dense(16, activation="tanh")(x)
z_mean = layers.Dense(latent_dim, name="z_mean")(x)
z_log_var = layers.Dense(latent_dim, name="z_log_var")(x)
z = Sampling()([z_mean, z_log_var])
encoder = keras.Model(encoder_inputs, [z_mean, z_log_var, z], name="encoder")

latent_inputs = keras.Input(shape=(latent_dim,))
# x = layers.Dense(7 * 7 * 64, activation="relu")(latent_inputs)
# x = layers.Reshape((7, 7, 64))(x)
# x = layers.Conv2DTranspose(64, 3, activation="relu", strides=2, padding="same")(x)
# x = layers.Conv2DTranspose(32, 3, activation="relu", strides=2, padding="same")(x)
# decoder_outputs = layers.Conv2DTranspose(1, 3, activation="sigmoid", padding="same")(x)
x = layers.Dense(16, activation="tanh")(latent_inputs)
x = layers.Dense(18, activation="tanh")(x)
x = layers.Dense(20, activation="tanh")(x)
decoder_outputs = layers.Dense(30, activation="tanh")(x)
decoder = keras.Model(latent_inputs, decoder_outputs, name="decoder")



class VAE(keras.Model):
    def __init__(self, encoder, decoder, **kwargs):
        super().__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder
        self.total_loss_tracker = keras.metrics.Mean(name="total_loss")
        self.reconstruction_loss_tracker = keras.metrics.Mean(
            name="reconstruction_loss"
        )
        self.kl_loss_tracker = keras.metrics.Mean(name="kl_loss")

    @property
    def metrics(self):
        return [
            self.total_loss_tracker,
            self.reconstruction_loss_tracker,
            self.kl_loss_tracker,
        ]

    def train_step(self, data):
        with tf.GradientTape() as tape:
            z_mean, z_log_var, z = self.encoder(data)
            reconstruction = self.decoder(z)
#             reconstruction_loss = tf.reduce_mean(
#                 tf.reduce_sum(
#                     keras.losses.binary_crossentropy(data, reconstruction), axis=(1, 2)
#                 )
#             )
            reconstruction_loss = tf.keras.losses.MeanSquaredError()(data,reconstruction)
            kl_loss = -0.5 * (1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var))
            kl_loss = tf.reduce_mean(tf.reduce_sum(kl_loss, axis=1))
            total_loss = reconstruction_loss + kl_loss
        grads = tape.gradient(total_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
        self.total_loss_tracker.update_state(total_loss)
        self.reconstruction_loss_tracker.update_state(reconstruction_loss)
        self.kl_loss_tracker.update_state(kl_loss)
        return {
            "loss": self.total_loss_tracker.result(),
            "reconstruction_loss": self.reconstruction_loss_tracker.result(),
            "kl_loss": self.kl_loss_tracker.result(),
        }

creditdata = np.concatenate([corrupted_data], axis=0)
creditdata = np.expand_dims(creditdata, -1).astype("float32")

vae = VAE(encoder, decoder)
vae.compile(optimizer=tf.keras.optimizers.legacy.Adam())
vae.fit(creditdata, epochs=100, batch_size=128)

Epoch 1/100


2023-06-23 15:49:22.184208: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-06-23 15:49:22.184309: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-06-23 15:49:22.184352: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-06-23 15:49:22.184517: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-06-23 15:49:22.184569: I tensorflow/compiler/xla/stream_executo

39/79 [=============>................] - ETA: 0s - loss: 1.5185 - reconstruction_loss: 1.1136 - kl_loss: 0.2269

2023-06-23 15:49:23.238387: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:630] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


79/79 [==============================] - 1s 4ms/step - loss: 1.4140 - reconstruction_loss: 1.1388 - kl_loss: 0.1270
Epoch 2/100
79/79 [==============================] - 0s 2ms/step - loss: 0.8697 - reconstruction_loss: 0.9449 - kl_loss: 0.0100
Epoch 3/100
79/79 [==============================] - 0s 1ms/step - loss: 0.9561 - reconstruction_loss: 0.9362 - kl_loss: 0.0043
Epoch 4/100
79/79 [==============================] - 0s 1ms/step - loss: 1.0993 - reconstruction_loss: 0.9984 - kl_loss: 0.0024
Epoch 5/100
79/79 [==============================] - 0s 1ms/step - loss: 0.8658 - reconstruction_loss: 0.9373 - kl_loss: 0.0016
Epoch 6/100
79/79 [==============================] - 0s 2ms/step - loss: 0.8523 - reconstruction_loss: 0.9368 - kl_loss: 0.0011
Epoch 7/100
79/79 [==============================] - 0s 5ms/step - loss: 0.8891 - reconstruction_loss: 0.9368 - kl_loss: 8.6242e-04
Epoch 8/100
79/79 [==============================] - 0s 4ms/step - loss: 0.9726 - reconstruction_loss: 0.9369 - 

79/79 [==============================] - 0s 5ms/step - loss: 0.8975 - reconstruction_loss: 0.9363 - kl_loss: 7.0996e-05
Epoch 63/100
79/79 [==============================] - 0s 5ms/step - loss: 0.9528 - reconstruction_loss: 0.9360 - kl_loss: 5.5822e-05
Epoch 64/100
79/79 [==============================] - 0s 5ms/step - loss: 1.0203 - reconstruction_loss: 0.9362 - kl_loss: 5.3477e-05
Epoch 65/100
79/79 [==============================] - 0s 5ms/step - loss: 1.0158 - reconstruction_loss: 0.9964 - kl_loss: 4.5147e-05
Epoch 66/100
79/79 [==============================] - 0s 6ms/step - loss: 1.0066 - reconstruction_loss: 0.9366 - kl_loss: 4.0295e-05
Epoch 67/100
79/79 [==============================] - 0s 3ms/step - loss: 1.0585 - reconstruction_loss: 0.9364 - kl_loss: 3.7704e-05
Epoch 68/100
79/79 [==============================] - 0s 5ms/step - loss: 0.9333 - reconstruction_loss: 0.9519 - kl_loss: 2.3562e-05
Epoch 69/100
79/79 [==============================] - 0s 5ms/step - loss: 0.9019 -

The error calculation for setting the threshold

In [ ]:
inliers = corrupted_data[5:80,:]
mean_data = np.mean(corrupted_data, axis=0)
outlier1 = corrupted_data[101,:]



# #------------------ replacing the genes here ---------------------

# inliers[:,1] = mean_data[1]

# outlier1[1] = mean_data[1]

# # ----------------------------------------------------------------

ouliers_mean_error = []

for i in range(20):
    
    mean_mean = []
    
    for step in range(10):
        
        outlier1 = corrupted_data[100 + i,:]
        
        outlier1[10] = mean_data[10]

        outlier1 = outlier1.reshape([1,30])

        z_mean, z_log_var, z = vae.encoder(outlier1)
        reconstruction = vae.decoder(z)

        reconstruction_loss = tf.keras.losses.MeanSquaredError()(outlier1,reconstruction)

#         print(reconstruction_loss.numpy())

        mean_mean.append(reconstruction_loss.numpy())

#     print("MEAN_MEAN = ", i, np.mean(np.array(mean_mean)))
#     print("std_MEAN = ", i, np.std(np.array(mean_mean)))
    
    ouliers_mean_error.append(np.mean(np.array(mean_mean)))

print(ouliers_mean_error)
print(np.mean(np.array(ouliers_mean_error)))

In [ ]:
mean_mean = []

for step in range(10):

    mean_ins_error = []

    for index in range(inliers.shape[0]):

        cand = inliers[index,:]

        cand = cand.reshape([1,30])

        mean_data = mean_data.reshape([1,30])

        z_mean, z_log_var, z = vae.encoder(cand)
        reconstruction = vae.decoder(z)

        reconstruction_loss = tf.keras.losses.MeanSquaredError()(cand,reconstruction)

        mean_ins_error.append(reconstruction_loss.numpy())

    #     print("MEAN", index, " = ", reconstruction_loss.numpy())

    mean_ins_error = np.array(mean_ins_error)
    print("mean = ", np.mean(mean_ins_error))
    
    mean_mean.append(np.mean(mean_ins_error))
    

print("MEAN_MEAN = ", np.mean(np.array(mean_mean)))
print("std_MEAN = ", np.std(np.array(mean_mean)))

Setting the threshold and trying to detect the outliers:

In [ ]:
for item_no in range(corrupted_data.shape[0]):
    
    candidate_sample = corrupted_data[item_no,:]
    candidate_sample = candidate_sample.reshape([1,30])
    
    z_mean, z_log_var, z = vae.encoder(candidate_sample)
    reconstruction = vae.decoder(z)

    reconstruction_loss = tf.keras.losses.MeanSquaredError()(candidate_sample,reconstruction)
    
    if reconstruction_loss.numpy() > 1:
        
        print("An Outlier Detected on sample No. ", item_no)

# Genetic Part

In [ ]:
def fitness_func_avg(ga_instance, solution, solution_idx):
    
    inliers = corrupted_data[10:13,:]
    
    avg_ins = np.mean(inliers, axis=0)
    avg_ins = avg_ins.reshape([1,30])
    
    particle = corrupted_data[701,:]
    particle = particle.reshape([1,30])
    
#     abn_subspace = solution * val_features[6728,:]
    
#     abn_subspace = abn_subspace.reshape([1,30])

    avg_in_rec = []
    
    for index in range(inliers.shape[0]):
        
        candidate_inlier = inliers[index,:]
        candidate_inlier = candidate_inlier.reshape([1,30])
        
        in_remain = candidate_inlier * solution
        
        in_normal_subspace = 1 - solution
        
        in_replace = in_normal_subspace * avg_ins
        
        in_candidate = in_remain + in_replace
        
        z_mean, z_log_var, z = vae.encoder(in_candidate)
        in_candidate_rec = vae.decoder(z)
        
        
        rec_loss = tf.keras.losses.MeanSquaredError()(in_candidate,in_candidate_rec)
        
        avg_in_rec.append(rec_loss.numpy())
    
    avg_in_rec = np.array(avg_in_rec)
    avg_in_rec = np.mean(avg_in_rec)
        
        
    
    

#     z_mean, z_log_var, z = vae.encoder(particle)
#     reconstruction_1 = vae.decoder(z)
    
    out_remain = particle * solution
    
    out_normal_subspace = 1 - solution
    
    out_replace = avg_ins * out_normal_subspace
    
    out_candidate = out_remain + out_replace
    
    
    z_mean, z_log_var, z = vae.encoder(out_candidate)
    out_candidate_rec = vae.decoder(z)
    
    rec_loss = tf.keras.losses.MeanSquaredError()(out_candidate,out_candidate_rec)
    rec_loss = rec_loss.numpy()
    
    fitness = rec_loss / avg_in_rec
    
    return fitness


def fitness_func_avg_with_penalty(ga_instance, solution, solution_idx):
    
    inliers = corrupted_data[10:13,:]
    
    avg_ins = np.mean(inliers, axis=0)
    avg_ins = avg_ins.reshape([1,30])
    
    particle = corrupted_data[201,:]
    particle = particle.reshape([1,30])
    
#     abn_subspace = solution * val_features[6728,:]
    
#     abn_subspace = abn_subspace.reshape([1,30])

    avg_in_rec = []
    
    for index in range(inliers.shape[0]):
        
        candidate_inlier = inliers[index,:]
        candidate_inlier = candidate_inlier.reshape([1,30])
        
        in_remain = candidate_inlier * solution
        
        in_normal_subspace = 1 - solution
        
        in_replace = in_normal_subspace * avg_ins
        
        in_candidate = in_remain + in_replace
        
        z_mean, z_log_var, z = vae.encoder(in_candidate)
        in_candidate_rec = vae.decoder(z)
        
        
        rec_loss = tf.keras.losses.MeanSquaredError()(in_candidate,in_candidate_rec)
        
        avg_in_rec.append(rec_loss.numpy())
    
    avg_in_rec = np.array(avg_in_rec)
    avg_in_rec = np.mean(avg_in_rec)
        
        
    
    

#     z_mean, z_log_var, z = vae.encoder(particle)
#     reconstruction_1 = vae.decoder(z)
    
    out_remain = particle * solution
    
    out_normal_subspace = 1 - solution
    
    out_replace = avg_ins * out_normal_subspace
    
    out_candidate = out_remain + out_replace
    
    
    z_mean, z_log_var, z = vae.encoder(out_candidate)
    out_candidate_rec = vae.decoder(z)
    
    rec_loss = tf.keras.losses.MeanSquaredError()(out_candidate,out_candidate_rec)
    rec_loss = rec_loss.numpy()
    
    fitness = rec_loss / avg_in_rec
    
    # adding the penalty here!
    
    num_ones = np.count_nonzero(solution == 1)
    
    penalty_rate = 1.0/corrupted_data.shape[1]
    
    fitness = fitness - num_ones * penalty_rate * fitness
    
    return fitness

def on_generation(ga):
    print("Generation", ga.generations_completed)
    
    solution, solution_fitness, solution_idx = ga_instance.best_solution()
    
    print(solution_fitness)


fitness_function = fitness_func_avg_with_penalty

num_generations = 20
num_parents_mating = 4

sol_per_pop = 100
num_genes = corrupted_data.shape[1]

init_range_low = -2
init_range_high = 5

parent_selection_type = "sss"
keep_parents = 1

space = [[0,1] for i in range(num_genes)]

crossover_type = "single_point"

mutation_type = "random"
mutation_percent_genes = 10

ga_instance = pygad.GA(num_generations=num_generations,
                       num_parents_mating=num_parents_mating,
                       fitness_func=fitness_function,
                       sol_per_pop=sol_per_pop,
                       num_genes=num_genes,
                       init_range_low=init_range_low,
                       init_range_high=init_range_high,
                       parent_selection_type=parent_selection_type,
                       keep_parents=keep_parents,
                       crossover_type=crossover_type,
                       mutation_type=mutation_type,
                       mutation_percent_genes=mutation_percent_genes,
                       on_generation=on_generation,
                       gene_space = space)

In [ ]:
ga_instance.run()

In [ ]:
solution, solution_fitness, solution_idx = ga_instance.best_solution()
print("Parameters of the best solution : {solution}".format(solution=solution))
print("Fitness value of the best solution = {solution_fitness}".format(solution_fitness=solution_fitness))

In [ ]:
ga_instance.plot_fitness()

# For loop for Genetic Algorithms

In [4]:
best_solutions = []

wandb.init(
    # set the wandb project where this run will be logged
    project="VAE_Outlier_GA_With_Penalty_1",
    
    # track hyperparameters and run metadata
#     config={
#     "learning_rate": 0.02,
#     "architecture": "CNN",
#     "dataset": "CIFAR-100",
#     "epochs": 20,
#     }
)

for out_indexing in range(1,corrupted_data.shape[1]+1):
    
#     def fitness_func_avg(ga_instance, solution, solution_idx):

#         inliers = corrupted_data[10:13,:]

#         avg_ins = np.mean(inliers, axis=0)
#         avg_ins = avg_ins.reshape([1,30])

#         particle = corrupted_data[700+out_indexing,:]
#         particle = particle.reshape([1,30])

#     #     abn_subspace = solution * val_features[6728,:]

#     #     abn_subspace = abn_subspace.reshape([1,30])

#         avg_in_rec = []

#         for index in range(inliers.shape[0]):

#             candidate_inlier = inliers[index,:]
#             candidate_inlier = candidate_inlier.reshape([1,30])

#             in_remain = candidate_inlier * solution

#             in_normal_subspace = 1 - solution

#             in_replace = in_normal_subspace * avg_ins

#             in_candidate = in_remain + in_replace

#             z_mean, z_log_var, z = vae.encoder(in_candidate)
#             in_candidate_rec = vae.decoder(z)


#             rec_loss = tf.keras.losses.MeanSquaredError()(in_candidate,in_candidate_rec)

#             avg_in_rec.append(rec_loss.numpy())

#         avg_in_rec = np.array(avg_in_rec)
#         avg_in_rec = np.mean(avg_in_rec)





#     #     z_mean, z_log_var, z = vae.encoder(particle)
#     #     reconstruction_1 = vae.decoder(z)

#         out_remain = particle * solution

#         out_normal_subspace = 1 - solution

#         out_replace = avg_ins * out_normal_subspace

#         out_candidate = out_remain + out_replace


#         z_mean, z_log_var, z = vae.encoder(out_candidate)
#         out_candidate_rec = vae.decoder(z)

#         rec_loss = tf.keras.losses.MeanSquaredError()(out_candidate,out_candidate_rec)
#         rec_loss = rec_loss.numpy()

#         fitness = rec_loss / avg_in_rec

#         return fitness
    
    def fitness_func_avg_with_penalty(ga_instance, solution, solution_idx):
    
        inliers = corrupted_data[10:13,:]

        avg_ins = np.mean(inliers, axis=0)
        avg_ins = avg_ins.reshape([1,30])

        particle = corrupted_data[700+out_indexing,:]
        particle = particle.reshape([1,30])

    #     abn_subspace = solution * val_features[6728,:]

    #     abn_subspace = abn_subspace.reshape([1,30])

        avg_in_rec = []

        for index in range(inliers.shape[0]):

            candidate_inlier = inliers[index,:]
            candidate_inlier = candidate_inlier.reshape([1,30])

            in_remain = candidate_inlier * solution

            in_normal_subspace = 1 - solution

            in_replace = in_normal_subspace * avg_ins

            in_candidate = in_remain + in_replace

            z_mean, z_log_var, z = vae.encoder(in_candidate)
            in_candidate_rec = vae.decoder(z)


            rec_loss = tf.keras.losses.MeanSquaredError()(in_candidate,in_candidate_rec)

            avg_in_rec.append(rec_loss.numpy())

        avg_in_rec = np.array(avg_in_rec)
        avg_in_rec = np.mean(avg_in_rec)





    #     z_mean, z_log_var, z = vae.encoder(particle)
    #     reconstruction_1 = vae.decoder(z)

        out_remain = particle * solution

        out_normal_subspace = 1 - solution

        out_replace = avg_ins * out_normal_subspace

        out_candidate = out_remain + out_replace


        z_mean, z_log_var, z = vae.encoder(out_candidate)
        out_candidate_rec = vae.decoder(z)

        rec_loss = tf.keras.losses.MeanSquaredError()(out_candidate,out_candidate_rec)
        rec_loss = rec_loss.numpy()

        # adding the penalty here!

        num_ones = np.count_nonzero(solution == 1)

        penalty_rate = 1.0 / corrupted_data.shape[1]
        
        regul_param = 1
        
        penalty_rate = penalty_rate / regul_param
        
        fitness = rec_loss / (avg_in_rec + num_ones * penalty_rate * avg_in_rec)

#         fitness = fitness - num_ones * penalty_rate * fitness

        return fitness

    def on_generation(ga):
        print("Generation", ga.generations_completed)

        solution, solution_fitness, solution_idx = ga_instance.best_solution()
        
        wandb.log({"GA - " + str(out_indexing) + "/solution_fitness": solution_fitness})

        print(solution_fitness)


    fitness_function = fitness_func_avg_with_penalty

    num_generations = 40
    num_parents_mating = 4

    sol_per_pop = 100
    num_genes = corrupted_data.shape[1]

    init_range_low = -2
    init_range_high = 5

    parent_selection_type = "sss"
    keep_parents = 1

    space = [[0,1] for i in range(num_genes)]

    crossover_type = "single_point"

    mutation_type = "random"
    mutation_percent_genes = 15

    ga_instance = pygad.GA(num_generations=num_generations,
                           num_parents_mating=num_parents_mating,
                           fitness_func=fitness_function,
                           sol_per_pop=sol_per_pop,
                           num_genes=num_genes,
                           init_range_low=init_range_low,
                           init_range_high=init_range_high,
                           parent_selection_type=parent_selection_type,
#                            keep_parents=keep_parents,
                           keep_elitism=2,
                           crossover_type=crossover_type,
                           mutation_type=mutation_type,
                           mutation_percent_genes=mutation_percent_genes,
                           on_generation=on_generation,
                           gene_space = space)

    ga_instance.run()

    solution, solution_fitness, solution_idx = ga_instance.best_solution()
    print("##########  End of the ", out_indexing, " epoch ##########")
    print("Parameters of the best solution : {solution}".format(solution=solution))

    best_solutions.append(solution)
    
    sample = corrupted_data[700+out_indexing,:]
    
    TP = 0
    FP = 0
    TN = 0
    FN = 0
    
    for gene_no in range(30):
        
        if ((sample[gene_no] == 20) and (solution[gene_no] == 1)):
            
            TP = TP + 1
        
        elif (sample[gene_no] != 20) and (solution[gene_no] == 1):
            
            FP = FP + 1
            
        elif (sample[gene_no] != 20) and (solution[gene_no] == 0):
            
            TN = TN + 1
            
        elif (sample[gene_no] == 20) and (solution[gene_no] == 0):
            
            FN = FN + 1
    
    precision = TP / (TP + FP)

    recall = TP / (TP + FN)

    F1 = 2 * (precision * recall)/(precision + recall)
    
    MCC = (TP * TN - FP * FN) / np.sqrt((TP + FP) * (TP + FN) * (TN + FP) * (TN + FN))
    
    wandb.log({"Metrics/precision": precision, 
               "Metrics/recall": recall,
               "Metrics/F1": F1,
               "Metrics/MCC": MCC,
               "Metrics/num": out_indexing})
    
best_solutions = np.array(best_solutions)
np.save('best_solutions.npy', best_solutions)

wandb: Currently logged in as: stmmc (tousi-team). Use `wandb login --relogin` to force relogin


Generation 1
227.4142902526143
Generation 2
236.33833294176154
Generation 3
254.21283850485653
Generation 4
257.04359956635636
Generation 5
262.99137013668667
Generation 6
263.67012929941075
Generation 7
263.67012929941075
Generation 8
263.67012929941075
Generation 9
263.67012929941075
Generation 10
267.31901588918174
Generation 11
267.3451335593387
Generation 12
271.03702143313467
Generation 13
278.17790818397395
Generation 14
278.17790818397395
Generation 15
278.17790818397395
Generation 16
278.17790818397395
Generation 17
278.17790818397395
Generation 18
278.17790818397395
Generation 19
278.17790818397395
Generation 20
278.17790818397395
Generation 21
278.17790818397395
Generation 22
278.17790818397395
Generation 23
278.17790818397395
Generation 24
278.17790818397395
Generation 25
278.17790818397395
Generation 26
278.17790818397395
Generation 27
278.17790818397395
Generation 28
278.17790818397395
Generation 29
278.17790818397395
Generation 30
278.17790818397395
Generation 31
278.177

1200.178205709289
Generation 31
1229.4022769665444
Generation 32
1249.3677562315725
Generation 33
1249.3677562315725
Generation 34
1249.3677562315725
Generation 35
1269.1638132165965
Generation 36
1278.2539308328803
Generation 37
1278.2539308328803
Generation 38
1278.2539308328803
Generation 39
1278.2539308328803
Generation 40
1278.2539308328803
##########  End of the  6  epoch ##########
Parameters of the best solution : [1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.
 0. 0. 0. 0. 0. 0.]
Generation 1
1065.3269090387591
Generation 2
1152.5847483013688
Generation 3
1170.9081199947234
Generation 4
1243.8703542145254
Generation 5
1221.2501322794367
Generation 6
1225.0571958843186
Generation 7
1253.1147044834934
Generation 8
1253.1147044834934
Generation 9
1344.0885095306721
Generation 10
1297.6803013695232
Generation 11
1297.6803013695232
Generation 12
1297.8917283255405
Generation 13
1313.0317754381258
Generation 14
1313.0317754381258
Generation 15
1313.031775438

Generation 11
1872.1668246424024
Generation 12
1870.305476404959
Generation 13
1870.305476404959
Generation 14
1870.305476404959
Generation 15
1888.6240609226986
Generation 16
1870.305476404959
Generation 17
1870.305476404959
Generation 18
1870.305476404959
Generation 19
1870.305476404959
Generation 20
1870.305476404959
Generation 21
1920.0477519228045
Generation 22
1894.6510634243073
Generation 23
1894.6510634243073
Generation 24
1894.6510634243073
Generation 25
1894.6510634243073
Generation 26
1894.6510634243073
Generation 27
1894.6510634243073
Generation 28
1894.6510634243073
Generation 29
1894.6510634243073
Generation 30
1894.6510634243073
Generation 31
1894.6510634243073
Generation 32
1894.6510634243073
Generation 33
1894.6510634243073
Generation 34
1894.6510634243073
Generation 35
1894.6510634243073
Generation 36
1894.6510634243073
Generation 37
1894.6510634243073
Generation 38
1894.6510634243073
Generation 39
1894.6510634243073
Generation 40
1894.6510634243073
##########  End of

Generation 36
2234.2235271583254
Generation 37
2234.2235271583254
Generation 38
2234.2235271583254
Generation 39
2234.2235271583254
Generation 40
2234.2235271583254
##########  End of the  17  epoch ##########
Parameters of the best solution : [0. 1. 1. 1. 1. 0. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 1. 0. 0.
 0. 0. 0. 0. 0. 0.]
Generation 1
1999.430467745303
Generation 2
2042.8064855393368
Generation 3
2092.704434317963
Generation 4
2112.370979221222
Generation 5
2171.272813384288
Generation 6
2145.1874781390547
Generation 7
2145.1874781390547
Generation 8
2145.1874781390547
Generation 9
2152.1889493806543
Generation 10
2192.64562648627
Generation 11
2294.021833988923
Generation 12
2294.021833988923
Generation 13
2294.021833988923
Generation 14
2294.021833988923
Generation 15
2294.021833988923
Generation 16
2294.021833988923
Generation 17
2294.021833988923
Generation 18
2294.021833988923
Generation 19
2294.021833988923
Generation 20
2294.021833988923
Generation 21
2294.021833988

2489.4402224105565
Generation 18
2489.4402224105565
Generation 19
2489.4402224105565
Generation 20
2489.4402224105565
Generation 21
2489.4402224105565
Generation 22
2489.4402224105565
Generation 23
2489.4402224105565
Generation 24
2489.4402224105565
Generation 25
2489.4402224105565
Generation 26
2489.4402224105565
Generation 27
2489.4402224105565
Generation 28
2489.4402224105565
Generation 29
2489.4402224105565
Generation 30
2489.4402224105565
Generation 31
2489.4402224105565
Generation 32
2489.4402224105565
Generation 33
2503.280165088406
Generation 34
2489.4402224105565
Generation 35
2524.337689412118
Generation 36
2524.337689412118
Generation 37
2524.337689412118
Generation 38
2524.337689412118
Generation 39
2524.337689412118
Generation 40
2524.337689412118
##########  End of the  23  epoch ##########
Parameters of the best solution : [0. 1. 1. 0. 1. 1. 0. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 0. 1. 1. 1. 1. 0. 0.
 0. 0. 0. 0. 0. 0.]
Generation 1
2334.3111355412802
Generation 2
2356.6373680

Generation 1
2404.3228956156954
Generation 2
2536.123346813404
Generation 3
2563.3502750074254
Generation 4
2575.540729234056
Generation 5
2579.3497277681677
Generation 6
2584.3827738526084
Generation 7
2592.862338723237
Generation 8
2606.6059331944434
Generation 9
2626.0253220000372
Generation 10
2604.17032071846
Generation 11
2584.3827738526084
Generation 12
2616.4067701121808
Generation 13
2584.3827738526084
Generation 14
2686.6214998777473
Generation 15
2594.154412994962
Generation 16
2631.4141134901984
Generation 17
2644.5277124345607
Generation 18
2644.5277124345607
Generation 19
2660.314708509372
Generation 20
2660.314708509372
Generation 21
2660.314708509372
Generation 22
2660.314708509372
Generation 23
2660.314708509372
Generation 24
2660.314708509372
Generation 25
2660.314708509372
Generation 26
2683.203377169865
Generation 27
2665.173567518247
Generation 28
2665.173567518247
Generation 29
2665.173567518247
Generation 30
2665.173567518247
Generation 31
2667.8623140479895
Gene

/home/macula/SMATousi/.conda/envs/mac-deep/lib/python3.7/site-packages/ipykernel_launcher.py:248: RuntimeWarning: invalid value encountered in double_scalars
wandb: Network error (ConnectTimeout), entering retry loop.


In [ ]:
corrupted_data[700+0,:]

# Measurements

F1 calculator

In [ ]:
TP = 3
FP = 1
TN = 22
FN = 4

precision = TP / (TP + FP)

recall = TP / (TP + FN)

F1 = 2 * (precision * recall)/(precision + recall)

MCC = (TP * TN - FP * FN) / np.sqrt((TP + FP) * (TP + FN) * (TN + FP) * (TN + FN))

print("Precision = ", precision)
print("recall = ", recall)
print("F1 = ", F1)
print("MCC = ", MCC)

In [ ]:
for index in range(corrupted_data.shape[1]):
    print("^^^^^^^^^^^^^^^^^^^^^^^^^^^^^")
    print(best_solutions[index,:])
    print(corrupted_data[700+index,:])
    print("######################################3")

In [ ]:
corrupted_data[701,:]

In [ ]:
precs = []
recalls = []
F1s = []
mccs = []


for sample_no in range(1,corrupted_data.shape[1]):
    
    TP = 0
    FP = 0
    TN = 0
    FN = 0
    
    sample = corrupted_data[700+sample_no,:]
    sol = best_solutions[sample_no,:]
    
    for gene_no in range(30):
        
        if ((sample[gene_no] == 20) and (sol[gene_no] == 1)):
            
            TP = TP + 1
        
        elif (sample[gene_no] != 20) and (sol[gene_no] == 1):
            
            FP = FP + 1
            
        elif (sample[gene_no] != 20) and (sol[gene_no] == 0):
            
            TN = TN + 1
            
        elif (sample[gene_no] == 20) and (sol[gene_no] == 0):
            
            FN = FN + 1
    
    

    precision = TP / (TP + FP)

    recall = TP / (TP + FN)

    F1 = 2 * (precision * recall)/(precision + recall)
    
    MCC = (TP * TN - FP * FN) / np.sqrt((TP + FP) * (TP + FN) * (TN + FP) * (TN + FN))

    print("###################################################3")
    print(sol)
    print(sample)
    print("Numbers = ", TP, FP, TN, FN)
    print("Precision = ", precision)
    print("recall = ", recall)
    print("F1 = ", F1)
    print("MCC = ", MCC)
    
    precs.append(precision)
    recalls.append(recall)
    F1s.append(F1)
    mccs.append(MCC)


precs = np.array(precs)
recalls = np.array(recalls)
F1s = np.array(F1s)
mccs = np.array(mccs)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
# precs = np.array(precs)
# recalls = np.array(recalls)
# F1s = np.array(F1s)
# mccs = np.array(mccs)

plt.title("Matthews correlation coefficient (MCC)")
plt.plot(mccs)
plt.xlabel("Number of outlying genes")
plt.savefig("MCC.png")

In [ ]:
plt.title("Precision")
plt.plot(precs)
plt.xlabel("Number of outlying genes")
plt.savefig("Precision.png")

In [ ]:
plt.title("Recall")
plt.plot(recalls)
plt.xlabel("Number of outlying genes")
plt.savefig("Recall.png")

In [ ]:
plt.title("F1")
plt.plot(F1s)
plt.xlabel("Number of outlying genes")
plt.savefig("F1.png")